# Neural Network

This notebook contains the procedure and decisions taken to create a neural network using [PyTorch](https://pytorch.org/) for the classification of the obesity level of patients based on their lifestyle habits.

In section [1. Load Dataset](#1-load-dataset) the training and evaluation datasets of the model are loaded making use of the functionalities offered by PyTorch.

In section [2. Model Architecture](#2-model-architecture) the neural network for the classifier is defined, with a focus on being a simple and small network but capable of learning to classify obesity levels correctly.

Finally, in section [3. Model Training](#3-model-training) the parameters (weights and bias) of the model defined in the previous section are optimized using the optimizer [Adam](https://docs.pytorch.org/docs/stable/generated/torch.optim.Adam.html) and [CrossEntropyLoss](https://docs.pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) as loss function.

# 1. Load Dataset

Subclassing of [`Dataset`](https://docs.pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) is used to load the dataset so that it can be compatible with the neural networks in PyTorch. The [`DataLoader`](https://docs.pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) used to train and evaluate the models are created in the `Trainer`. When loading the dataset, no normalization or scaling transformation is applied to the dataset.

In [1]:
# Importing auxiliar libraries

import marimo as mo

# Importing libraries

from functools import partial
from torch import nn , device , optim , Tensor
from torch.cuda import is_available

import pandas as pd
from torch.utils.data import Dataset

# Importing Functions and Utils

import SourceModels as src

In [2]:
# Defining useful variables

PATH = './'
PATH_SAVE = PATH + 'SaveModels/'

NUM_JOBS = src.GetNumJobs()
TORCH_DEVICE = device('cuda' if is_available() else 'cpu')

RANDOM_STATE = 8013

In [3]:
# Splitting features 

DatasetFilename = PATH + 'Dataset_{}.csv'
_Dataset = pd.read_csv(DatasetFilename.format('Train'),nrows=1)

NumericalFeatures , CategoricalFeatures , Target = src.SplitFeatures(_Dataset)
Features = [*NumericalFeatures,*CategoricalFeatures]

del _Dataset

In [4]:
# Loading datasets

Dataset_Train: Dataset = None
Dataset_Evaluation: Dataset = None
for _type_dataset in ['Train','Evaluation']:
    globals()[f'Dataset_{_type_dataset}'] = src.DatasetLoader(
        DatasetFilename.format(_type_dataset),
        Features,
        Target,
    )

# 2. Model Architecture

It makes use of a simple architecture with two hidden layers of $40$ and $25$ neurons, respectively, with `ReLU` activation functions.

In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        """
        Neural network architecture for 
        predicting the obesity level of 
        a person
        """

        super().__init__()

        self.NN = nn.Sequential(
            nn.Linear(21,40),
            nn.ReLU(),
            nn.Linear(40,25),
            nn.ReLU(),
            nn.Linear(25,7),
        )

    def forward(
            self,
            Instance_X: Tensor
        ) -> Tensor:

        Logits = self.NN(Instance_X)
        return Logits

# 3. Model Training

For the training of the neural network it was decided to use the optimizer [Adam](https://docs.pytorch.org/docs/stable/generated/torch.optim.Adam.html), due to its robustness generated by its adaptive learning mechanisms, with a learning rate of $0.01$ and $50$ epochs with batch size of $32$ because there are not enough training instances (this setting was made by means of tests and empirical rules).

For determining the best model, the F1 metric with weighted average is used, due to the slight imbalance of the dataset with respect to the target (`NObeyesdad`), as described in the [Exploratory Data Analysis](../ExploratoryDataAnalysis/ExploratoryDataAnalysis.ipynb). Finally, the best model obtained during training is saved.

In [6]:
# Defining optimizer, loss function and metric

_LearningRate = 1e-2
Optimizer = partial(optim.Adam,lr=_LearningRate)

LossFunction = nn.CrossEntropyLoss()
MetricFunction = src.F1_NN()

In [7]:
# Initializing trainer of Neural Network

NN_Trainer = src.NeuralNetworTrainer(
        NeuralNetwork,
        Optimizer,
        LossFunction,
    )

In [8]:
# Training of neural network

_Epochs = 50
BatchSize = 32
BestModel = NN_Trainer(
    Dataset_Train,
    Dataset_Evaluation,
    BatchSize,
    _Epochs,
    MetricFunction,
    TORCH_DEVICE
)

-------- Epoch 1 --------
Loss :: 3.106110  [   32/ 1350]
Loss :: 1.897667  [  352/ 1350]
Loss :: 1.844657  [  672/ 1350]
Loss :: 1.766185  [  992/ 1350]
Loss :: 1.600338  [ 1312/ 1350]
Test Error: 
Score: 17.6%, Avg loss: 1.597215 

-------- Epoch 2 --------
Loss :: 1.542831  [   32/ 1350]
Loss :: 1.337827  [  352/ 1350]
Loss :: 1.352473  [  672/ 1350]
Loss :: 1.383006  [  992/ 1350]
Loss :: 1.178139  [ 1312/ 1350]
Test Error: 
Score: 48.0%, Avg loss: 1.172976 

-------- Epoch 3 --------
Loss :: 1.128736  [   32/ 1350]
Loss :: 1.122481  [  352/ 1350]
Loss :: 0.976173  [  672/ 1350]
Loss :: 1.212871  [  992/ 1350]
Loss :: 0.991295  [ 1312/ 1350]
Test Error: 
Score: 59.6%, Avg loss: 0.927245 

-------- Epoch 4 --------
Loss :: 0.973949  [   32/ 1350]
Loss :: 1.019642  [  352/ 1350]
Loss :: 1.013087  [  672/ 1350]
Loss :: 1.016852  [  992/ 1350]
Loss :: 0.911138  [ 1312/ 1350]
Test Error: 
Score: 61.7%, Avg loss: 0.853728 

-------- Epoch 5 --------
Loss :: 1.026529  [   32/ 1350]
Loss :

In [9]:
# Saving best model

# src.SaveModelNN(BestModel,PATH_SAVE,'NN')